### SILVER LAYER SCRIPT

# Silver Layer Transformation – Credit Risk Dataset

This notebook mounts the bronze and silver containers, loads raw CSV,
cleans missing values, and saves the cleaned data in the Silver zone as Parquet.


### DATA LOADING

In [ ]:
from pyspark.sql.functions import col


In [ ]:
storage_account_name = "creditriskdataset"
bronze_container = "bronze"
bronze_sas_token = "<your_bronze_sas_token>"

# Unmount if already exists
try:
    dbutils.fs.unmount(f"/mnt/{bronze_container}")
except:
    pass

# Mount the bronze container
dbutils.fs.mount(
    source=f"wasbs://{bronze_container}@{storage_account_name}.blob.core.windows.net",
    mount_point=f"/mnt/{bronze_container}",
    extra_configs={
        f"fs.azure.sas.{bronze_container}.{storage_account_name}.blob.core.windows.net": bronze_sas_token
    }
)



NameError: name 'dbutils' is not defined

In [ ]:
df = spark.read.csv(
    "/mnt/bronze/credit-risk/credit_risk_dataset.csv",
    header=True,
    inferSchema=True
)
display(df)




In [ ]:
from pyspark.sql.functions import col

# Drop rows with missing values in important columns
df_cleaned = df.dropna(subset=["person_emp_length", "loan_int_rate"])

# Cast target column to integer if needed
df_cleaned = df_cleaned.withColumn("loan_status", col("loan_status").cast("int"))

display(df_cleaned)



In [ ]:
silver_container = "silver"
silver_sas_token = "<your_silver_sas_token>"

# Unmount if already exists
try:
    dbutils.fs.unmount(f"/mnt/{silver_container}")
except:
    pass

# Mount the silver container
dbutils.fs.mount(
    source=f"wasbs://{silver_container}@{storage_account_name}.blob.core.windows.net",
    mount_point=f"/mnt/{silver_container}",
    extra_configs={
        f"fs.azure.sas.{silver_container}.{storage_account_name}.blob.core.windows.net": silver_sas_token
    }
)


In [ ]:
df_cleaned.write.mode("overwrite").parquet("/mnt/silver/credit_risk_cleaned")
